In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, preprocessing
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define paths to your dataset directories
train_data_dir = '/kaggle/input/skindata/train'
test_data_dir = '/kaggle/input/skindata/test'

# Set image size and parameters
img_width, img_height = 150, 150
batch_size = 32  
learning_rate = 0.0001 

# Create ImageDataGenerators without data augmentation, only rescaling
train_datagen = preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)

# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42
)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Data for the bar chart
labels = ['benign', 'malignant']
training_data = [1400, 1200]
test_data = [360, 300]

# Create the bar chart
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 8))

# First bar chart for training data
ax1.bar(labels, training_data, color=['blue', 'goldenrod'])
ax1.set_ylabel('Number of Images')
ax1.set_xlabel('Skin Condition')
ax1.set_title('Training Data Class Distribution')
ax1.grid(True, axis='y')

# Second bar chart for test data
ax2.bar(labels, test_data, color=['blue', 'goldenrod'])
ax2.set_ylabel('Number of Images')
ax2.set_xlabel('Skin Condition')
ax2.set_title('Test Data Class Distribution')
ax2.grid(True, axis='y')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plot
plt.show()

# Display the first 8 images from the training dataset
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images and labels from the training generator
train_images, train_labels = next(train_generator)

plt.figure(figsize=(16, 8))  # Set the figure size
for i in range(8):
    plt.subplot(2, 4, i + 1)  # Arrange in 2 rows and 4 columns
    plt.imshow(train_images[i])
    label = 'Benign' if train_labels[i] == 0 else 'Malignant'
    plt.title(label, color='black', fontsize=14)
    plt.axis('off')  # Hide axes

plt.tight_layout()
plt.suptitle("Training Dataset - First 8 Images", fontsize=16, color='black', y=1.05)
plt.show()

# Display the first 8 images from the test dataset
test_images, test_labels = next(test_generator)

plt.figure(figsize=(16, 8))  # Set the figure size
for i in range(8):
    plt.subplot(2, 4, i + 1)  # Arrange in 2 rows and 4 columns
    plt.imshow(test_images[i])
    label = 'Benign' if test_labels[i] == 0 else 'Malignant'
    plt.title(label, color='black', fontsize=14)
    plt.axis('off')  # Hide axes

plt.tight_layout()
plt.suptitle("Test Dataset - First 8 Images", fontsize=16, color='black', y=1.05)
plt.show()

# Calculate class weights
class_counts = np.bincount(train_generator.classes)
weight_for_benign_class = class_counts[0] / sum(class_counts)
weight_for_malignant_class = class_counts[1] / sum(class_counts)

class_weight = {0: weight_for_benign_class, 1: weight_for_malignant_class}

# Convert generators to TensorFlow datasets for pre-fetching and caching
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()

# Define the learning rate scheduler callback

learning_rate_scheduler = ReduceLROnPlateau(
    monitor='val_loss',    # Metric to monitor
    factor=0.5,            # Factor by which the learning rate will be reduced
    patience=3,            # Number of epochs with no improvement to wait before reducing the learning rate
    min_lr=1e-6            # Minimum learning rate
)

# Load the ResNet50 model with pre-trained weights
resnet_base = keras.applications.ResNet50(
    weights='imagenet',  
    include_top=False,   
    input_shape=(img_width, img_height, 3)  
)

# Freeze the ResNet50 base model layers
resnet_base.trainable = False

# Build the model
model = models.Sequential([
    resnet_base,  
    layers.GlobalAveragePooling2D(),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.L2(0.001)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.L2(0.001)),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define the EarlyStopping and modelcheckpoint callback
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_checkpoint = keras.callbacks.ModelCheckpoint(
    'best_model.keras', monitor='val_loss', save_best_only=True
)

# Train the model
history = model.fit(
    train_dataset,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,
    validation_data=test_dataset,
    validation_steps=test_generator.samples // batch_size,
    class_weight=class_weight,
    callbacks=[early_stopping, model_checkpoint, learning_rate_scheduler]
)

# Save the model after the initial training phase
model.save('initial_trained_model.keras')

The model is saved. Now, Restart and Clear Cell Outputs to free the RAM for fine tuning

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# Define paths to your dataset directories
train_data_dir = '/kaggle/input/skindata/train'
test_data_dir = '/kaggle/input/skindata/test'

img_width, img_height = 150, 150
batch_size = 32  # Adjust as necessary
learning_rate = 0.0001

# Create ImageDataGenerators without data augmentation, only rescaling
train_datagen = preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)
test_datagen = preprocessing.image.ImageDataGenerator(rescale=1.0 / 255.0)

# Load the training dataset
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True,
    seed=42
)

# Load the test dataset
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# Calculate class weights
class_counts = np.bincount(train_generator.classes)
weight_for_benign_class = class_counts[0] / sum(class_counts)
weight_for_malignant_class = class_counts[1] / sum(class_counts)

class_weight = {0: weight_for_benign_class, 1: weight_for_malignant_class}

# Convert generators to TensorFlow datasets for pre-fetching and caching
train_dataset = tf.data.Dataset.from_generator(
    lambda: train_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()

test_dataset = tf.data.Dataset.from_generator(
    lambda: test_generator,
    output_signature=(
        tf.TensorSpec(shape=(None, img_width, img_height, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).cache()

# Load the model saved after the first training phase
model = keras.models.load_model('initial_trained_model.keras')

# Re-define the ResNet50 base model 
resnet_base = keras.applications.ResNet101( 
    weights='imagenet',
    include_top=False,
    input_shape=(img_width, img_height, 3)
)

# Unfreeze some layers in the ResNet101 model for fine-tuning
for layer in resnet_base.layers[-40:]:  # Unfreezing last 40 layers
    layer.trainable = True

# Add custom layers to increase model complexity
model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),  # Pooling layer for feature extraction
    layers.Dense(512, activation='relu'),  # Increased number of units for more complexity
    layers.Dropout(0.4),  # Dropout to avoid overfitting
    layers.Dense(256, activation='relu'),  # Another dense layer
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')  # Output layer
])

# Recompile the model after adding custom layers
model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate / 10),  
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Fine-tune the model with the newly unfrozen layers
history_fine_tune = model.fit(
    train_dataset,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=25,  
    validation_data=test_dataset,
    validation_steps=test_generator.samples // batch_size,
    class_weight=class_weight
)

# Evaluate the model on the test data
test_steps = test_generator.samples // batch_size
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_steps)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


In [ ]:
# Plot training history
plt.plot(history_fine_tune.history['accuracy'], label='Train Accuracy')
plt.plot(history_fine_tune.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

plt.plot(history_fine_tune.history['loss'], label='Train Loss')
plt.plot(history_fine_tune.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
import seaborn as sns

# Get predictions from the test set
y_pred = model.predict(test_generator, steps=int(np.ceil(test_generator.samples / batch_size)), verbose=1)

# Convert the probabilities to binary class predictions (0 or 1)
y_pred_class = (y_pred > 0.5).astype(int)

# Ensure the number of predictions matches the number of true labels
y_pred_class = y_pred_class[:test_generator.samples]

# Print classification report
print(classification_report(test_generator.classes, y_pred_class))

# Confusion Matrix
cm = confusion_matrix(test_generator.classes, y_pred_class)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

# ROC-AUC Curve
y_true = test_generator.classes
y_pred_prob = y_pred.flatten()  # Get the probability for the positive class (class 1)

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)
roc_auc = roc_auc_score(y_true, y_pred_prob)

# Plot ROC curve
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.show()


In [ ]:
# Function to plot saliency maps
def plot_saliency_map(model, image, true_label):
    image = tf.convert_to_tensor(image)
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(tf.expand_dims(image, axis=0))
    
    # Calculate the gradient of the top predicted class with respect to the input image
    grads = tape.gradient(prediction, image)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1))
    
    # Get the weights for the saliency map
    saliency_map = tf.reduce_mean(tf.multiply(pooled_grads, image), axis=-1)

    # Plot the image and its saliency map
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image.numpy())
    plt.title(f'True Label: {true_label}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(saliency_map.numpy(), cmap='hot')
    plt.title('Saliency Map')
    plt.axis('off')
    plt.show()

# Example to plot saliency map for a test image
test_image, true_label = next(test_generator)  # Use the built-in `next()` function
plot_saliency_map(model, test_image[0], true_label[0])


In [ ]:
# Get a batch of test data and predictions
test_generator.reset()  # Reset the generator to ensure it starts from the beginning
batch = next(test_generator)  # Fetch the first batch of test data
images, true_labels = batch
predicted_probs = model.predict(images, verbose=1)
predicted_labels = (predicted_probs > 0.5).astype(int)  # Convert probabilities to binary predictions

# Map class indices to labels
class_indices = {v: k for k, v in test_generator.class_indices.items()}
predicted_class_names = [class_indices[label[0]] for label in predicted_labels]
true_class_names = [class_indices[label] for label in true_labels]

# Plot the images with predicted and true labels
plt.figure(figsize=(16, 12))
for i in range(len(images)):
    plt.subplot(4, 4, i + 1)  # Create a grid of 4x4 (adjust if batch size is larger)
    plt.imshow(images[i])
    plt.axis('off')
    plt.title(f"True: {true_class_names[i]}\nPred: {predicted_class_names[i]}")
    if i == 15:  # Display only the first 16 images
        break
plt.tight_layout()
